In [1]:
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
import boto3
import sys
import re
from datetime import datetime
import configparser

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1664874632079_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
conf = SparkConf()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.config(
            conf=conf).enableHiveSupport().getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
Q400Final = spark.sql('''
SELECT * FROM precogdb.q400128wps WHERE filename = 'ALL ac=VT-SUB dep=IXB arr=CCU fn=SG 9223 dt=20220629.csv'
''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
Q400Final.createOrReplaceTempView('Q400Final')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Q400_stage3_df = Q400Final.filter(~(F.col("Radio-Altitude").like ("*")) 
                                                        & (F.col("Radio-Altitude") != " "))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
Q400Interim = Q400Final.alias('ED').select(
                                                        'ED.FlightID',
                                                        'ED.DesigFlightNumber',
                                                        'ED.FlightNumberARMS',
                                                        'ED.FlightDate',
                                                        'ED.FromCity',
                                                        'ED.ToCity',
                                                        'ED.BlockTime',
                                                        'ED.ECN',
                                                        'ED.EmployeeName',
                                                        'ED.RANK',
                                                        'ED.ChocksOff',
                                                        'ED.ChocksOn',
                                                        'ED.DOMINT',
                                                        'ED.AircraftType',
                                                        'ED.AircraftModel',  
                                                        'ED.Payload',
                                                        'ED.TailNo',
                                                        'ED.Departure',
                                                        'ED.Arrival',
                                                        'ED.FlightNo',
                                                        'ED.Date',
                                                         F.regexp_replace(
                                                               F.col('ED.Time-In-Secs'), '\\*', '').cast('Double').alias('Time-In-Secs'),
                                                        'ED.Time-Per-Sec', 
                                                        'ED.BARO-CORRECTED-PALT-DERIVED', 
                                                        'ED.Altitude-BaroL', 
                                                        'ED.Magnetic-Heading', 
                                                        F.lit(F.regexp_replace(
                                                        'ED.Pitch-Attitude', '\\*', '')).alias('Pitch-Attitude'),
                                                        'ED.Roll-Attitude',  
                                                        F.col('ED.Radio-Altitude').cast('Integer').alias('Radio-Altitude'),
                                                        F.lit(F.regexp_replace('ED.Nose-WOW', '\\*', '')).alias('Nose-WOW'), 
                                                        F.lit(F.regexp_replace('ED.Main-WOW', '\\*', '')).alias('Main-WOW'),
                                                        'ED.Normal-Acceleration', 
                                                        'ED.Lateral-Acceleration', 
                                                        'ED.Longitudinal-Acceleration', 
                                                        'ED.Calibrated-Airspeed-L', 
                                                        'ED.AFCS-Selected-Speed', 
                                                        'ED.Vertical-Speed-Derived', 
                                                        'ED.Ground-Speed', 
                                                        'ED.Flap-Control-Selection',
                                                        F.regexp_replace(F.col('ED.FLAP-POSITION'), '\\*', '').alias('FLAP-POSITION'), 
                                                        'ED.Power-Lever-Pos-L', 
                                                        'ED.Power-Lever-Pos-R', 
                                                        'ED.Torque-TQ-Eng-L', 
                                                        'ED.Torque-TQ-Eng-R', 
                                                        'ED.Condition-Lever-L', 
                                                        'ED.Condition-Lever-R', 
                                                        'ED.Prop-RPM-Np-Eng-L', 
                                                        'ED.Prop-RPM-Np-Eng-R', 
                                                        'ED.GPWS-Alert/Warning', 
                                                        'ED.GPWS-Glideslope-Alert', 
                                                        F.regexp_replace(F.col('ED.AFCS-Glideslope-Active'), '\\*', '').alias("AFCS-Glideslope-Active"),
                                                        'ED.AFCS-LOC-Active', 
                                                        'ED.AFCS-VOR-Active', 
                                                        F.regexp_replace(F.col('ED.GLIDE-PATH-DEV-PILOT-ABOVE'), '\\*', '').alias("GLIDE-PATH-DEV-PILOT-ABOVE"),  
                                                        F.regexp_replace(F.col('ED.GLIDE-PATH-DEV-COPILOT-ABOVE'), '\\*', '').alias("GLIDE-PATH-DEV-COPILOT-ABOVE"),
                                                        'ED.DME-DISTANCE-1', 
                                                        'ED.DME-DISTANCE-2', 
                                                        'ED.VOR-1-SELECT-FREQ', 
                                                        'ED.VOR-2-SELECT-FREQ', 
                                                        'ED.WIND-ANGLE', 
                                                        'ED.WIND-SPEED', 
                                                        'ED.BRAKE-PRESS-LEFT', 
                                                        'ED.BRAKE-PRESS-RIGHT', 
                                                        'ED.Thrust-Rev-Low-Beta-Eng-L', 
                                                        'ED.Thrust-Rev-Low-Beta-Eng-R', 
                                                        'ED.AFCS-Autopilot-Engaged',
                                                        'ED.FileName',
                                                        'ED.LoadedDate',
                                                        'ED.LoadedBy',
                                                        'ED.DataFrameName',
                                                        'ED.year',
                                                        'ED.month' )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
Q400Interim.createOrReplaceTempView('Q400Interim')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
spark.sql('''select distinct(filename) from Q400Interim ''').show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+
|filename                                                |
+--------------------------------------------------------+
|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|ALL ac=VT-SUB dep=BOM arr=DEL fn=SEJ7402 dt=20220313.csv|
+--------------------------------------------------------+

In [9]:
FirstQuery  = spark.sql('''SELECT FileName, max(cast(`Radio-Altitude` as int)) as RadioAltitude from Q400Final group by FileName''')
    
FirstQuery.createOrReplaceTempView("B")

SecondQuery = spark.sql('''SELECT a.*, B.RadioAltitude from Q400Final a inner join B on a.FileName = B.FileName and a.`Radio-Altitude` = B.RadioAltitude''')

SecondQuery.createOrReplaceTempView("C")

ThirdQuery = spark.sql('''SELECT FileName, RadioAltitude, max(`Time-In-Secs`) as MaxTime, min(`Time-In-Secs`) as MinTime from C group by FileName, RadioAltitude''')

ThirdQuery.createOrReplaceTempView("D")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
ThirdQuery.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+-------------+-------+-------+
|FileName                                                |RadioAltitude|MaxTime|MinTime|
+--------------------------------------------------------+-------------+-------+-------+
|ALL ac=VT-SUB dep=IXB arr=CCU fn=SG 9223 dt=20220629.csv|2492         |764.0  |764.0  |
+--------------------------------------------------------+-------------+-------+-------+

In [15]:
FourthQuery = spark.sql(f'''

                                        SELECT * FROM 
                                        
                                        (
                                        
                                        select a1.`Time-In-Secs` as `Time-In-Secs-EX`,
                                        
                                        a1.`BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                                        
                                        a1.`Radio-Altitude` as `Radio-Altitude-EX`,
                                        
                                        REPLACE(a1.`FLAP-POSITION`,'*','') as `FLAP-POSITION-EX`,
                                        
                                        a1.`Main-WOW`,
                                        
                                        'LateLandingFlap' as ExceedanceName,
                                        
                                        a1.filename,
                                        
                                        case 
                                        
                                        when Cast(`Time-In-Secs` as int)<D.MinTime Then 'TakeOff'
                                        
                                        when Cast(`Time-In-Secs` as int) between D.MinTime and D.MaxTime Then 'Airborne'
                                        
                                        when Cast(`Time-In-Secs` as int)>D.MaxTime Then 'Approach' end Phase
                                        
                                        ,ROW_NUMBER() over(partition by a1.filename order by `Time-In-Secs`) RN
                                        
                                        from Q400Interim a1
                                        
                                        inner join D  
                                        
                                        on a1.FileName = D.FileName
                                        
                                        where Cast(`Time-In-Secs` as float)>D.MaxTime 
                                        
                                        and `Radio-Altitude` < 1000
                                        
                                        and `Radio-Altitude` <> 0 
                                        
                                        and `FLAP-POSITION` IS NOT NULL 
                                        
                                        and `Main-WOW` = 'Air (0)' 
                                        
                                        )
                                        
                                        WHERE RN = 1 
                                        
                                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
FourthQuery.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------+--------+---+
|Time-In-Secs-EX|BARO-CORRECTED-PALT-DERIVED-EX|Radio-Altitude-EX|FLAP-POSITION-EX|Main-WOW| ExceedanceName|            filename|   Phase| RN|
+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------+--------+---+
|          796.0|                          3819|                3|            -0.1| Air (0)|LateLandingFlap|ALL ac=VT-SUB dep...|Approach|  1|
+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------+--------+---+

In [129]:
FifthQuery.createOrReplaceTempView('E')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

TESTING

In [163]:
FinalDF = spark.sql(f'''
            
            SELECT 
                    *
            FROM
                    (
                        SELECT 
                                `Time-In-Secs` as `Time-In-Secs-EX`,
                                `Radio-Altitude` AS `Radio-Altitude-EX` ,
                                `Main-WOW` AS `Main-WOW-EX`,
                                `Pitch-Attitude` AS `Pitch-Attitude-EX`,
                                'PitchAttitudeHigh'  as ExceedanceName,
                                DE.Phase,
                                CASE
                                    WHEN
                                            CASE
                                            WHEN
                                                    `Pitch-Attitude` LIKE 'U%'
                                            THEN
                                                    CASE
                                                    WHEN
                                                            INSTR(`Pitch-Attitude`,' ')>0
                                                    THEN
                                                            SUBSTRING(`Pitch-Attitude`, INSTR(`Pitch-Attitude`,' ')+1, LENGTH(`Pitch-Attitude`))
                                                    ELSE
                                                            NULL
                                                    END
                                            END                     > 10.0 
                                    THEN
                                            'High'
                                ELSE
                                        'Low'
                                END TriggerCondition,
                                BC.*
                                
                                            
                                            
                        FROM
                                Q400Interim BC
                        
                        INNER JOIN



                                (
                                    SELECT 
                                            FileName,
                                            Phase,
                                            MAX(`Time-In-Secs`) AS TriggerTime
                                    FROM 
                                        (
                                            SELECT 

                                                    CASE
                                                        WHEN
                                                                `Time-In-Secs` < DJ.MinTime
                                                        THEN
                                                                'TakeOff'
                                                        WHEN
                                                                `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                        THEN
                                                                'Airborne'
                                                        WHEN
                                                                `Time-In-Secs` >DJ.MaxTime
                                                        THEN
                                                                'Approach'
                                                        END Phase,              

                                                    BJ.*

                                            FROM Q400Interim AS BJ

                                            INNER JOIN(

                                    SELECT 

                                    BI.FileName,
                                    BI.`Radio-Altitude`,
                                    MIN(`Time-In-Secs`) as MinTime,
                                    MAX(`Time-In-Secs`) as MaxTime


                                    FROM Q400Interim AS BI
                                    INNER JOIN 
                                        (
                                            SELECT 
                                                    FileName,
                                                    MAX(`Radio-Altitude`) AS MaxRadioHeight
                                            FROM Q400Interim

                                            GROUP BY FileName) T1 
                                    ON BI.FileName=T1.FileName 
                                    AND BI.`Radio-Altitude`=T1.MaxRadioHeight

                                    GROUP BY 
                                        BI.FileName,
                                        BI.`Radio-Altitude` ) DJ 
                        ON BJ.FileName=DJ.FileName ) DD

                WHERE DD.Phase='TakeOff'
                AND    DD.`Main-WOW` = 'Gnd (1)'
                
                GROUP BY
                        DD.FileName,
                        DD.Phase )DE
                ON
                        BC.FileName    = DE.FileName
                AND     BC.`Time-In-Secs` =DE.TriggerTime) FNN
                                                            ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [164]:
FinalDF.where(F.col('FileName') == 'ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+-----------------+-----------------+-------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+------------+-----+--------------------+--------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----

In [168]:
spark.sql('''

SELECT 
                        FileName,
                        Phase,
                        MAX(CAST(`Time-In-Secs` AS FLOAT)) AS TriggerTime
                        FROM(
                                            SELECT 

                                                    CASE
                                                        WHEN
                                                                `Time-In-Secs` < DJ.MinTime
                                                        THEN
                                                                'TakeOff'
                                                        WHEN
                                                                `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                        THEN
                                                                'Airborne'
                                                        WHEN
                                                                `Time-In-Secs` >DJ.MaxTime
                                                        THEN
                                                                'Approach'
                                                        END Phase,              

                                                    BJ.*

                                            FROM Q400Interim AS BJ

                                            INNER JOIN(

                                    SELECT 

                                    BI.FileName,
                                    BI.`Radio-Altitude`,
                                    MIN(`Time-In-Secs`) as MinTime,
                                    MAX(`Time-In-Secs`) as MaxTime


                                    FROM Q400Interim AS BI
                                    INNER JOIN 
                                        (
                                            SELECT 
                                                    FileName,
                                                    MAX(`Radio-Altitude`) AS MaxRadioHeight
                                            FROM Q400Interim

                                            GROUP BY FileName) T1 
                                    ON BI.FileName=T1.FileName 
                                    AND BI.`Radio-Altitude`=T1.MaxRadioHeight

                                    GROUP BY 
                                        BI.FileName,
                                        BI.`Radio-Altitude` ) DJ 
                        ON BJ.FileName=DJ.FileName ) DD
                    WHERE DD.Phase='TakeOff'
                    AND    DD.`Main-WOW` = 'Gnd (1)'
                    
                    GROUP BY FileName,
                                Phase
                                            ''').show(truncate=False)
#.where((F.col('FileName') == 'ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv')).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+-------+-----------+
|FileName                                                |Phase  |TriggerTime|
+--------------------------------------------------------+-------+-----------+
|ALL ac=VT-SUB dep=BOM arr=DEL fn=SEJ7402 dt=20220313.csv|TakeOff|771.0      |
|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|TakeOff|1193.0     |
|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|TakeOff|628.0      |
+--------------------------------------------------------+-------+-----------+

In [166]:
FinalDF.show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+-----------------+-----------------+-------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+------------------------+------------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------

In [132]:
spark.sql('''

                        SELECT 
                                `Time-In-Secs` as `Time-In-Secs-EX`,
                                `Radio-Altitude` AS `Radio-Altitude-EX` ,
                                `Main-WOW` AS `Main-WOW-EX`,
                                `Pitch-Attitude` AS `Pitch-Attitude-EX`,
                                'PitchAttitudeHigh'  as ExceedanceName,
                                DE.Phase,
                                CASE
                                    WHEN
                                            CASE
                                            WHEN
                                                    `Pitch-Attitude` LIKE 'U%'
                                            THEN
                                                    CASE
                                                    WHEN
                                                            INSTR(`Pitch-Attitude`,' ')>0
                                                    THEN
                                                            SUBSTRING(`Pitch-Attitude`, INSTR(`Pitch-Attitude`,' ')+1, LENGTH(`Pitch-Attitude`))
                                                    ELSE
                                                            NULL
                                                    END
                                            END                     > 10.0 
                                    THEN
                                            'High'
                                    ELSE
                                            'Low'
                                    END TriggerCondition,
                                    BC.*
                                            
                                            
                        FROM
                                Q400Interim BC
                        
                        INNER JOIN



                                (
                                    SELECT 
                                            FileName,
                                            Phase,
                                            MAX(`Time-In-Secs`) AS TriggerTime
                                    FROM 
                                        (
                                            SELECT 

                                                    CASE
                                                        WHEN
                                                                `Time-In-Secs` < DJ.MinTime
                                                        THEN
                                                                'TakeOff'
                                                        WHEN
                                                                `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                        THEN
                                                                'Airborne'
                                                        WHEN
                                                                `Time-In-Secs` >DJ.MaxTime
                                                        THEN
                                                                'Approach'
                                                        END Phase,              

                                                    BJ.*

                                            FROM Q400Interim AS BJ

                                            INNER JOIN(

                                    SELECT 

                                    BI.FileName,
                                    BI.`Radio-Altitude`,
                                    MIN(`Time-In-Secs`) as MinTime,
                                    MAX(`Time-In-Secs`) as MaxTime


                                    FROM Q400Interim AS BI
                                    INNER JOIN 
                                        (
                                            SELECT 
                                                    FileName,
                                                    MAX(`Radio-Altitude`) AS MaxRadioHeight
                                            FROM Q400Interim

                                            GROUP BY FileName) T1 
                                    ON BI.FileName=T1.FileName 
                                    AND BI.`Radio-Altitude`=T1.MaxRadioHeight

                                    GROUP BY 
                                        BI.FileName,
                                        BI.`Radio-Altitude` ) DJ 
                        ON BJ.FileName=DJ.FileName ) DD

                WHERE DD.Phase='TakeOff'
                AND    DD.`Main-WOW` = 'Gnd (1)'
                
                GROUP BY
                        DD.FileName,
                        DD.Phase )DE
                ON
                        BC.FileName    = DE.FileName
                AND     BC.`Time-In-Secs` = DE.TriggerTime
''').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+-----------------+-----------------+-------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+--------------------+--------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-

In [177]:
spark.sql('''

                                    SELECT 
                                            FileName,
                                            MAX(`Time-In-Secs`) AS TriggerTime
                                    FROM 
                                        (
                                            SELECT 

                                                    CASE
                                                        WHEN
                                                                `Time-In-Secs` < DJ.MinTime
                                                        THEN
                                                                'TakeOff'
                                                        WHEN
                                                                `Time-In-Secs` between DJ.MinTime and DJ.MaxTime
                                                        THEN
                                                                'Airborne'
                                                        WHEN
                                                                `Time-In-Secs` >DJ.MaxTime
                                                        THEN
                                                                'Approach'
                                                        END Phase,              

                                                    BJ.*

                                            FROM Q400Interim AS BJ

                                            INNER JOIN(

                                    SELECT 

                                    BI.FileName,
                                    BI.`Radio-Altitude`,
                                    MIN(`Time-In-Secs`) as MinTime,
                                    MAX(`Time-In-Secs`) as MaxTime


                                    FROM Q400Interim AS BI
                                    INNER JOIN 
                                        (
                                            SELECT 
                                                    FileName,
                                                    MAX(`Radio-Altitude`) AS MaxRadioHeight
                                            FROM Q400Interim

                                            GROUP BY FileName) T1 
                                    ON BI.FileName=T1.FileName 
                                    AND BI.`Radio-Altitude`=T1.MaxRadioHeight

                                    GROUP BY 
                                        BI.FileName,
                                        BI.`Radio-Altitude` ) DJ 
                        ON BJ.FileName=DJ.FileName ) DD

                WHERE DD.Phase='TakeOff'
                AND    DD.`Main-WOW` = 'Gnd (1)'
                
                GROUP BY FileName
                        ''').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+-----------+
|FileName                                                |TriggerTime|
+--------------------------------------------------------+-----------+
|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|628        |
|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|480        |
|ALL ac=VT-SUB dep=BOM arr=DEL fn=SEJ7402 dt=20220313.csv|85         |
+--------------------------------------------------------+-----------+

In [174]:
spark.sql(''' SELECT MAX(CAST(`Time-In-Secs` AS DOUBLE))
                FROM Q400Interim 
                WHERE FileName='ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv'
                AND CAST(`Time-In-Secs` AS DOUBLE) < 900
               AND `Main-WOW` = 'Gnd (1)' ''').show(truncate=False) 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------+
|max(CAST(Time-In-Secs AS DOUBLE))|
+---------------------------------+
|730.0                            |
+---------------------------------+

In [170]:
spark.sql(''' SELECT `Time-In-Secs`,`Radio-Altitude`,`Main-Wow`,`Pitch-Attitude`,FileName
                FROM Q400Interim 
                WHERE FileName='ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv'
                AND `Time-In-Secs` > 725 ''').show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------------+--------+--------------+--------------------------------------------------------+
|Time-In-Secs|Radio-Altitude|Main-Wow|Pitch-Attitude|FileName                                                |
+------------+--------------+--------+--------------+--------------------------------------------------------+
|726         |4             |Gnd (1) |D 0.5         |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|727         |3             |Gnd (1) |D 0.4         |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|728         |4             |Gnd (1) |D 0.3         |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|729         |4             |Gnd (1) |U 1.3         |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|730         |6             |Gnd (1) |U 13.8        |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|731         |8             |Air (0) |U 5.8         |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|

In [109]:
FinalDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+-----------+-----------------+--------------+-----+----------------+--------+-----------------+----------------+----------+--------+------+---------+---+------------+----+---------+--------+------+------------+-------------+-------+------+---------+-------+--------+----+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+-----------------+-----------------+------

In [83]:
FifthQuery = spark.sql(f'''SELECT * FROM E WHERE TriggerCondition = 'High' 
                        and `Radio-Altitude` < '1000' 
                        and `Phase` = 'TakeOff' ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
FifthQuery.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+-------------------------+--------------+-----+----------------+--------+-----------------+----------------+----------+--------+------+---------+---+------------+----+---------+--------+------+------------+-------------+-------+------+---------+-------+--------+----+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+---------------

In [57]:
FifthQuery = spark.sql('''

                SELECT * FROM E WHERE 
                TriggerCondition = 'High' 
                and cast(replace(`Vertical-Speed-Derived`,'*','') as int) > -300
                and `Radio-Altitude` < 1000
                and `Phase` = 'Approach'
                 
                ''')
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
FifthQuery.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+-------------------------+-----------------+-------------------+--------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+--------------------+--------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+------------

In [19]:
#FourthTestQuery.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
FifthQuery = spark.sql('''

                    SELECT * FROM 
                    
                    (
                    
                    select a1.`Time-In-Secs` as `Time-In-Secs-EX`,
                    
                    a1.`BARO-CORRECTED-PALT-DERIVED` as `BARO-CORRECTED-PALT-DERIVED-EX`,
                    
                    a1.`Radio-Altitude` as `Radio-Altitude-EX`,
                    
                    REPLACE(a1.`FLAP-POSITION`,'*','') as `FLAP-POSITION-EX`,
                    
                    a1.`Main-WOW`,
                    
                    'LateLandingFlap' as ExceedanceName,
                    
                    a1.filename,
                    
                    case 
                    
                    when Cast(`Time-In-Secs` as int)<D.MinTime Then 'TakeOff'
                    
                    when Cast(`Time-In-Secs` as int) between D.MinTime and D.MaxTime Then 'Airborne'
                    
                    when Cast(`Time-In-Secs` as int)>D.MaxTime Then 'Approach' end Phase,
                    
                    ROW_NUMBER() over(partition by a1.filename order by `Time-In-Secs`) RN
                    
                    from Q400Interim a1
                    
                    inner join D  
                    
                    on a1.FileName = D.FileName
                    
                    
                    where Cast(`Time-In-Secs` as float)>D.MaxTime 
                    
                    and `Radio-Altitude` < 1000 
                    
                    and `FLAP-POSITION` IS NOT NULL 
                    
                    and `Main-WOW` = 'Gnd (1)'
                    
                    )
                    
                    WHERE RN = 1
                    
                   ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
FifthQuery.createOrReplaceTempView('FifthQuery')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
FifthQuery.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------------------------------------------+--------+---+
|Time-In-Secs-EX|BARO-CORRECTED-PALT-DERIVED-EX|Radio-Altitude-EX|FLAP-POSITION-EX|Main-WOW|ExceedanceName |filename                                                |Phase   |RN |
+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------------------------------------------+--------+---+
|6600           |419                           |5                |35.3            |Gnd (1) |LateLandingFlap|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|Approach|1  |
|5038           |71                            |5                |35              |Gnd (1) |LateLandingFlap|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|Approach|1  |
|8800           |764                           |0                |14.9            |Gnd (1) |LateLandingFl

In [17]:
FifthTestQuery = spark.sql('''
                                SELECT
                            CASE 
                                WHEN `FLAP-POSITION-EX` IS NOT NULL THEN 
                                    CASE WHEN CAST((CAST(`FLAP-POSITION-EX` AS FLOAT)) AS INT)  BETWEEN 14 AND 16 THEN 15
                                         WHEN CAST((CAST(`FLAP-POSITION-EX` AS FLOAT)) AS INT)  BETWEEN 34 AND 36 THEN 35
                                         ELSE 35
                                    END
                            END FlapType,
                            FQ.FileName
                                            
                                           
             FROM FifthQuery FQ
                                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
FifthTestQuery.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------------------------------------------+
|FlapType|FileName                                                |
+--------+--------------------------------------------------------+
|35      |ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|35      |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|15      |ALL ac=VT-SUB dep=BOM arr=DEL fn=SEJ7402 dt=20220313.csv|
+--------+--------------------------------------------------------+

In [19]:
FifthTestQuery.createOrReplaceTempView('FifthTestQuery')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
FinalLLF = spark.sql('''
              SELECT * FROM (
              
              SELECT 
                  
                  QI.`Time-In-Secs` as `Time-In-Secs-EX`,
                  
                  QI.`Radio-Altitude` as `Radio-Altitude-EX`,
                  
                  FTF.FlapType,
                  
                  QI.`FLAP-POSITION` as `FLAP-POSITION-EX`,
                  
                  'LateLandingFlap'  as ExceedanceName,
                  
                  FQ.Phase,
                  
                  CASE 
                            WHEN QI.`FLAP-POSITION` IS NOT NULL THEN
                                 CASE 
                                     WHEN CAST(QI.`FLAP-POSITION` AS FLOAT) > CAST (FTF.FlapType AS Float) +1 
                                     OR CAST(QI.`FLAP-POSITION` AS FLOAT) < CAST (FTF.FlapType AS Float) -1  THEN 'High' ELSE 'Low'
                                    END
                        ELSE 'Low'
                        END AS TriggerCondition,
                        
                        QI.*
                        
                FROM Q400Interim QI
                
                INNER JOIN  FifthTestQuery FTF ON QI.FileName= FTF.FileName
                INNER JOIN  FourthQuery FQ ON QI.FileName= FQ.FileName
                                                    AND QI.`Radio-Altitude` = FQ.`Radio-Altitude-EX`
                                                    AND QI.`Time-In-Secs` = FQ.`Time-In-Secs-EX`
                                                    
                )FNL
                WHERE FNL.TriggerCondition ='High'
                
                ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
FinalLLF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----------------+--------+----------------+---------------+--------+----------------+--------+-----------------+----------------+----------+--------+------+---------+------+----------------+-----+--------------------+--------------------+------+------------+-------------+-------+------+---------+-------+--------+---------+------------+------------+---------------------------+--------------+----------------+--------------+-------------+--------------+--------+--------+-------------------+--------------------+-------------------------+---------------------+-------------------+----------------------+------------+----------------------+-------------+-----------------+-----------------+---------------+---------------+-----------------+-----------------+-----------------+-----------------+------------------+---------------------+----------------------+---------------+---------------+--------------------------+----------------------------+--------------+--------------+------

In [74]:
FifthQuery.show(5,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------------------------------------------+--------+---+
|Time-In-Secs-EX|BARO-CORRECTED-PALT-DERIVED-EX|Radio-Altitude-EX|FLAP-POSITION-EX|Main-WOW|ExceedanceName |filename                                                |Phase   |RN |
+---------------+------------------------------+-----------------+----------------+--------+---------------+--------------------------------------------------------+--------+---+
|6600.0         |419                           |5                |35.3            |Gnd (1) |LateLandingFlap|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|Approach|1  |
|5038.0         |71                            |5                |35.0            |Gnd (1) |LateLandingFlap|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|Approach|1  |
|8800.0         |764                           |0                |14.9            |Gnd (1) |LateLandingFl

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
SeventhQuery = SixthQuery.select('FileName').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
SeventhQuery.show(5, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------------------------------+
|FileName                                                |
+--------------------------------------------------------+
|ALL ac=VT-SUG dep=GOI arr=RAJ fn=SG 3702 dt=20220819.csv|
|ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
+--------------------------------------------------------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+
|FlapType|FileName|
+--------+--------+
+--------+--------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+------------+--------------+--------------------------------------------------------+
|Flap-Position|Main-WOW|Time-In-Secs|Radio-Altitude|FileName                                                |
+-------------+--------+------------+--------------+--------------------------------------------------------+
|null         |Gnd (1) |5315.0      |2             |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null         |Gnd (1) |5309.0      |3             |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null         |Gnd (1) |5251.0      |4             |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null         |Air (0) |5037.0      |6             |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null         |Air (0) |5035.0      |8             |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null         |Air (0) |5033.0      |17            |ALL ac=VT-SUD dep=BOM arr=IXY fn=SG 3323 dt=20220514.csv|
|null     